#Imports


In [127]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 21.0 MB/s eta 0:00:00


In [1]:
!pip install git+https://github.com/csebuetnlp/normalizer
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-ol0yh9ql
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-ol0yh9ql
  Resolved https://github.com/csebuetnlp/normalizer to commit d80c3c484e1b80268f2b2dfaf7557fe65e34f321
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6877 sha256=584487d9733ac0df0162a93ef96bde1a52377ec94c71e45679035b263c88cc1b
  Stored in directory: /tmp/pip-ephem-wheel-cache-2f90zo0k/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji

In [2]:
from normalizer import normalize
import torch
from transformers import ElectraTokenizer, ElectraForPreTraining, ElectraForTokenClassification, AdamW
from transformers import pipeline, AutoTokenizer, AutoModelForPreTraining , BertModel
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [3]:
from tqdm import tqdm

#Gazeteer import

In [4]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.entity_tags = ["PER", "LOC", "CW", "CORP", "GRP", "PROD"]
        self.tag_encoding = {"O": 0, "B-PER": 1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4,
                             "B-CW": 5, "I-CW": 6, "B-CORP": 7, "I-CORP": 8, 
                             "B-GRP": 9, "I-GRP": 10, "B-PROD": 11, "I-PROD": 12}

    def insert(self, word, entity_type):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
        node.entity_type = entity_type

    def search(self, sentence):
        encoding = [0] * len(sentence)
        for i in range(len(sentence)):
            node = self.root
            for j in range(i, len(sentence)):
                char = sentence[j]
                if char not in node.children:
                    break
                node = node.children[char]
                if node.is_end_of_word:
                    entity_type = node.entity_type
                    entity_length = j - i + 1
                    start_index = i
                    end_index = j
                    for k in range(start_index, end_index + 1):
                        if k == start_index:
                            encoding[k] = self.tag_encoding["B-" + entity_type]
                        else:
                            encoding[k] = self.tag_encoding["I-" + entity_type]
                    break
        one_hot_encoding = [[0] * 13 for i in range(len(encoding))]
        for i, tag in enumerate(encoding):
            one_hot_encoding[i][tag] = 1
        return one_hot_encoding

import numpy as np

def group_encodings_by_word(encoding, sentence):
    # Create an empty list to store the word encodings
    word_encodings = []
    
    # Create an empty list to store the current word encoding
    current_word_encoding = []
    
    # Create an empty string to store the current word
    current_word = ""
    
    # Iterate over each character encoding and character in the input encoding list and sentence, respectively
    for char_encoding, char in zip(encoding, sentence):
        # If the current character is a whitespace character, finish the current word and add its first character encoding to the word encodings list
        if char == " ":
            if len(current_word_encoding) > 0:
                word_encodings.append(np.array(current_word_encoding[0]))
                current_word_encoding = []
            current_word = ""
        # If the current character is part of a word, append the character encoding to the current word encoding and the character to the current word
        else:
            current_word_encoding.append(char_encoding)
            current_word += char
    
    # Add the last word encoding to the word encodings list, if it exists
    if len(current_word_encoding) > 0:
        word_encodings.append(np.array(current_word_encoding[0]))
    
    # Return the word encodings as a NumPy array
    return torch.from_numpy(np.array(word_encodings)).type(torch.float32)



def save_trie(trie, filename):
    with open(filename, "wb") as f:
        pickle.dump(trie, f)

def load_trie(filename):
    with open(filename, "rb") as f:
        trie = pickle.load(f)
    return trie


In [5]:
def Gtoken(text):
  inputs = tokenizer.encode_plus(text, return_tensors='pt')

  encoded_dict = tokenizer.encode_plus(
                  text,       # Sentence to encode.
                  add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                  max_length = 314,           # Pad & truncate all sentences.
                  padding = 'max_length',
                  return_attention_mask = True,   # Construct attn. masks.
                  return_tensors = 'pt',
                  truncation=False)
  input_ids = encoded_dict['input_ids']
  tokenized = tokenizer.convert_ids_to_tokens([i.item() for i in input_ids.squeeze() if i > 1])
  return "< "+" ".join(tokenized)+" >"

In [6]:
%cd /content/drive/MyDrive/Bracu/THESIS/Trie DS/

/content/drive/MyDrive/Bracu/THESIS/Trie DS


In [7]:
trie = load_trie('Trie final.bin')

In [8]:
model_name = '/content/drive/MyDrive/Thesis/BERTOUTPUT/checkpoint-11000/'
tokenizer = ElectraTokenizer.from_pretrained(model_name)

In [9]:
def pad_tensor(tensor):
    current_size = tensor.size(0)
    if current_size >= 64:
        return tensor[:64, :]  # if the tensor is larger than (64, 13), truncate it
    
    padded_tensor = torch.zeros((64, 13))
    padded_tensor[:current_size, :] = tensor  # copy the input tensor to the padded tensor
    return padded_tensor



In [19]:
def Gazetteer(sentence, pad = False):
  sentence = Gtoken(normalize(sentence))
  encoding = trie.search(sentence)
  tensor = group_encodings_by_word(encoding,sentence)
  if pad:
    current_size = tensor.size(0)
    if current_size >= 64:
        return tensor[:64, :]  # if the tensor is larger than (64, 13), truncate it
    
    padded_tensor = torch.zeros((64, 13))
    padded_tensor[:current_size, :] = tensor  # copy the input tensor to the padded tensor
    return padded_tensor
  else: 
    return tensor

In [11]:
Gazetteer("২০১৮ এর সেরা বর্ণানুক্রমিকভাবে তালিকাভুক্ত র‍্যাঙ্ক করা হয়নি এনপিআর", False)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

#Dataset Load

In [12]:
%cd /content/drive/MyDrive/Bracu/THESIS/DatasetSir

/content/drive/MyDrive/Bracu/THESIS/DatasetSir


In [49]:
# Load the NER dataset
df_train = pd.read_csv('trainData2022PP2.csv')
df_val = pd.read_csv('devData2022PP2.csv')
df_test = pd.read_csv('testData2022PP2.csv')

In [50]:
d = {'O': 0, 'B-CORP': 1, 'I-CORP': 2, 'B-CW': 3, 'I-CW': 4, 'B-GRP': 5, 'I-GRP': 6, 'B-LOC': 7, 'I-LOC': 8, 'B-PER': 9, 'I-PER': 10, 'B-PROD': 11, 'I-PROD': 12}
def label_encoder(x):
    x = eval(x)
    y = []
    for i in x:
      y.append(d[i])
    return y

def text_normalizer(x):
    return normalize(x)

df_train['Word'] = df_train['Word'].apply(lambda x: text_normalizer(x))
df_test['Word'] = df_test['Word'].apply(lambda x: text_normalizer(x))
df_val['Word'] = df_val['Word'].apply(lambda x: text_normalizer(x))
df_train['Tag'] = df_train['Tag'].apply(lambda x: label_encoder(x))
df_test['Tag'] = df_test['Tag'].apply(lambda x: label_encoder(x))
df_val['Tag'] = df_val['Tag'].apply(lambda x: label_encoder(x))
df_train.head()

,Sentence #,Word,Tag
0,Sentence: 1,২০১৮ এর সেরা বর্ণানুক্রমিকভাবে তালিকাভুক্ত র‍্...,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,Sentence: 10,সিনেমায় গানটির বৈশিষ্ট্য রয়েছে রাস্তা যাত্রা ।,"[0, 0, 0, 0, 3, 4, 0]"
2,Sentence: 100,করে বাদ্যযন্ত্রের থিম এবং চলচ্চিত্রের জন্য প্র...,"[5, 0, 0, 0, 0, 0, 0, 0]"
3,Sentence: 1000,প্রতিষ্ঠান ২২১১ ইঙ্গিত করে যে আউগুস্তুস পৃথক ক...,"[0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Sentence: 10000,শো স্টপারে তাদের বেকড আলাস্কা করতে সাড়ে চার ঘ...,"[0, 0, 0, 11, 12, 0, 0, 0, 0, 0, 0]"


In [51]:
def align_label(df):
  for row in tqdm(range(df.shape[0])):
    text = df['Word'][row]
    Tag = df['Tag'][row]
    inputs = tokenizer(text, return_tensors='pt')
    tokenized = tokenizer.convert_ids_to_tokens([i.item() for i in inputs['input_ids'].squeeze() if i > 1])
    i = 0
    j = 0
    new_tag = []
    prev_tag = 0
    while i < len(tokenized):
      if tokenized[i].startswith('['): 
        new_tag.append(0)
      elif tokenized[i].startswith('#'):
        if prev_tag % 2 != 0:
          new_tag.append(prev_tag+1)
        else:
          new_tag.append(prev_tag)
      else:
        if tokenized[i] == '.' or tokenized[i] == '′':
          new_tag.append(Tag[j])
          prev_tag = Tag[j]
        else:
          new_tag.append(Tag[j])
          prev_tag = Tag[j]
          j+=1
      i+=1
    df['Tag'][row] = new_tag
  return df


In [52]:
df_val = align_label(df_val)


100%|██████████| 801/801 [00:01<00:00, 590.82it/s]


In [53]:
df_train = align_label(df_train)

100%|██████████| 15301/15301 [00:20<00:00, 749.12it/s] 


In [137]:
df_test = align_label(df_test)

100%|██████████| 133114/133114 [02:20<00:00, 946.36it/s] 


In [54]:
def one_hot_encode_list(lst, pad = False):
    """
    One-hot encodes each element in a Python list and returns a tensor containing the one-hot encodings.
    
    Args:
    - lst: A Python list
    
    Returns:
    - A PyTorch tensor with shape (len(lst), 13), where each row represents the one-hot encoding of an element in the input list.
    """
    one_hot_tensors = torch.zeros(len(lst), 13)
    for i, elem in enumerate(lst):
        one_hot = torch.zeros(13)
        one_hot[elem] = 1
        one_hot_tensors[i] = one_hot
    if pad:
      tensor = one_hot_tensors
      current_size = tensor.size(0)
      if current_size >= 64:
          return tensor[:64, :]  # if the tensor is larger than (64, 13), truncate it
      
      padded_tensor = torch.zeros((64, 13))
      padded_tensor[:current_size, :] = tensor  # copy the input tensor to the padded tensor
      return padded_tensor
    return one_hot_tensors


one_hot_encode_list([0, 0, 0, 0, 3, 4, 0],False).size()

torch.Size([7, 13])

In [ ]:
# def tokenized_df(text):
#     inputs = tokenizer(text, max_length = 64, padding = 'max_length', return_tensors='pt')
#     input_ids = inputs['input_ids']
#     return input_ids

# def tokenized_df1(text):
#     inputs = tokenizer(text, max_length = 64, padding = 'max_length', return_tensors='pt')
#     attention_mask = inputs['attention_mask']
#     return attention_mask
# def tokenized_df2(text):
#     gazetteer = Gazetteer(text).unsqueeze(0)
#     return gazetteer

In [ ]:
# df_val['input_ids'] =  df_val['Word'].apply(lambda x: tokenized_df(x))
# df_val['attention_mask'] =  df_val['Word'].apply(lambda x: tokenized_df1(x))
# df_val['gazetteer'] =  df_val['Word'].apply(lambda x: tokenized_df2(x))
# df_train['input_ids'] = df_train['Word'].apply(lambda x: tokenized_df(x))
# df_train['attention_mask'] = df_train['Word'].apply(lambda x: tokenized_df1(x))
# df_train['gazetteer'] = df_train['Word'].apply(lambda x: tokenized_df2(x))

In [55]:
df_val['Tag'] =  df_val['Tag'].apply(lambda x: one_hot_encode_list(x))
df_train['Tag'] =  df_train['Tag'].apply(lambda x: one_hot_encode_list(x))

In [138]:
df_test['Tag'] =  df_test['Tag'].apply(lambda x: one_hot_encode_list(x))

In [56]:
df_train.head()

,Sentence #,Word,Tag
0,Sentence: 1,২০১৮ এর সেরা বর্ণানুক্রমিকভাবে তালিকাভুক্ত র‍্...,"[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
1,Sentence: 10,সিনেমায় গানটির বৈশিষ্ট্য রয়েছে রাস্তা যাত্রা ।,"[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
2,Sentence: 100,করে বাদ্যযন্ত্রের থিম এবং চলচ্চিত্রের জন্য প্র...,"[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
3,Sentence: 1000,প্রতিষ্ঠান ২২১১ ইঙ্গিত করে যে আউগুস্তুস পৃথক ক...,"[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
4,Sentence: 10000,শো স্টপারে তাদের বেকড আলাস্কা করতে সাড়ে চার ঘ...,"[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."


#Model

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [23]:
class NERClassification(nn.Module):
  def __init__(self, n_classes):
    super(NERClassification, self).__init__()
    self.bert = ElectraForTokenClassification.from_pretrained(model_name, output_hidden_states=True)
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.gOut = nn.Linear(n_classes,n_classes)
    self.final_out = nn.Linear(2*n_classes, n_classes)
    self.softmax = nn.Softmax(dim = 1)

  def forward(self, input_ids, attention_mask, gazetteer):
    pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    hidden_states = pooled_output.hidden_states

    # print(hidden_states)
    
    layer6 = hidden_states[6]
    layer12 = hidden_states[12]
    layer18 = hidden_states[18]


    averaged_hidden_states = torch.mean(torch.stack([layer6, layer12, layer18]), dim=0)

    # print(pooled_output)

    output = self.drop(averaged_hidden_states)
    output = self.out(output)
    goutput = self.gOut(gazetteer)
    final_output = torch.concat((output,goutput), dim = 2)
    final_output = self.final_out(final_output)
    final_output = self.softmax(final_output)
    return final_output

In [24]:
model = NERClassification(n_classes=13)

In [26]:
text = "সিনেমায় গানটির বৈশিষ্ট্য রয়েছে রাস্তা যাত্রা ।"
inputs = tokenizer(text, return_tensors='pt')


In [27]:
inputs['input_ids']

tensor([[    2, 13509, 20928,  7887,  1401,  2772,  5078,   205,     3]])

In [28]:
inputs['attention_mask'].size()

torch.Size([1, 9])

In [29]:
x = Gazetteer(text)

In [30]:
x

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [58]:


text = "সিনেমায় গানটির বৈশিষ্ট্য রয়েছে রাস্তা যাত্রা ।"
inputs = tokenizer(text, return_tensors='pt')

input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)
gazetteer = Gazetteer(text).unsqueeze(0).to(device)
# Run the model
outputs = model(input_ids=input_ids, attention_mask=attention_mask, gazetteer = gazetteer)

print(outputs.size())

# Get the predicted label
x, predicted_label = torch.max(outputs, dim=2)

predicted_label

# Print the predicted label
# print(predicted_label.item())

torch.Size([1, 9, 13])


tensor([[ 5, 12,  5, 11,  3,  0,  0,  3,  5]], device='cuda:0')

In [69]:
from sklearn.metrics import classification_report
print(classification_report(predicted_label[0].to('cpu'),predicted_label[0].to('cpu'), output_dict = True)['macro avg']['f1-score'])

1.0


In [46]:
print(classification_report([0,0,0,3,4,4],[0,0,0,4,4,5]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           3       0.00      0.00      0.00         1
           4       0.50      0.50      0.50         2
           5       0.00      0.00      0.00         0

    accuracy                           0.67         6
   macro avg       0.38      0.38      0.38         6
weighted avg       0.67      0.67      0.67         6



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [34]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()


NERClassification(
  (bert): ElectraForTokenClassification(
    (electra): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(32000, 1024, padding_idx=0)
        (position_embeddings): Embedding(512, 1024)
        (token_type_embeddings): Embedding(2, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0-23): 24 x ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): ElectraSelfOutput(
                (dense): Linea

In [116]:
from torch.nn import CrossEntropyLoss

loss1 = CrossEntropyLoss()
optim = AdamW(model.parameters(), lr=5e-5)
CS_SCORE = []

for epoch in range(10):
  c1 = []
  for i in tqdm(range((df_train.shape[0]))):
    text = df_train['Word'][i]
    label = df_train['Tag'][i].to(device)
    g = Gazetteer(df_train['Word'][i]).unsqueeze(0).to(device)
    inputs = tokenizer(text, return_tensors='pt')
    inp = inputs['input_ids'].to(device)
    a = inputs['attention_mask'].to(device)
    if inp.size()[1] != g.size()[1]:
      continue
    outputs = model(input_ids = inp, attention_mask=a, gazetteer = g)
    loss = loss1(outputs[0],label)
    if i % 100 == 0:
      optim.zero_grad()
    loss.backward()
    optim.step()
  print(loss)
  print("Running Test")
  mac_f1 = []
  jhamela = 0
  for i in tqdm(range((df_val.shape[0]))):
    text = df_val['Word'][i]
    label = df_val['Tag'][i].to(device)
    g = Gazetteer(df_val['Word'][i]).unsqueeze(0).to(device)
    inputs = tokenizer(text, return_tensors='pt')
    inp = inputs['input_ids'].to(device)
    a = inputs['attention_mask'].to(device)
    if inp.size()[1] != g.size()[1]:
      jhamela += 1
      continue
    outputs = model(input_ids = inp, attention_mask=a, gazetteer = g)
    _, predicted_label = torch.max(outputs, dim=2)
    _, real = torch.max(label, dim=1)
    c1.append(classification_report(predicted_label[0].to('cpu'),real.to('cpu'), output_dict = True, zero_division=0))
    mac_f1.append(classification_report(predicted_label[0].to('cpu'),real.to('cpu'), output_dict = True, zero_division=0)['macro avg']['f1-score'])
  print("MACRO F1 = ", sum(mac_f1) / (len(mac_f1)-jhamela))
  if epoch+1 % 3 == 0:
    with open('single model e{}.pkl'.format(epoch+1), 'wb') as f:
      pickle.dump(model, f)


model.eval()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 15301/15301 [40:16<00:00,  6.33it/s]


Running Test


100%|██████████| 801/801 [00:48<00:00, 16.48it/s]


MACRO F1 =  0.1285671379313281


 26%|██▌       | 3979/15301 [10:31<29:55,  6.30it/s]


KeyboardInterrupt: ignored

In [133]:
import torch
import torchmetrics

# Assuming y_true and y_pred are PyTorch tensors
y_true = torch.tensor([0, 1, 2, 0, 1, 2])
y_pred = torch.tensor([0, 2, 1, 0, 2, 1])

# Calculate F1 score using torchmetrics
f1 = torchmetrics.functional.f1_score(y_pred, y_true, task = 'multiclass', num_classes=13, average='micro', zero_division = 1)

print(f"F1 score: {f1:.3f}")

TypeError: ignored

In [140]:
for i in tqdm(range((df_test.shape[0]))):
    text = df_test['Word'][i]
    label = df_test['Tag'][i].to(device)
    g = Gazetteer(df_test['Word'][i]).unsqueeze(0).to(device)
    inputs = tokenizer(text, return_tensors='pt')
    inp = inputs['input_ids'].to(device)
    a = inputs['attention_mask'].to(device)
    if inp.size()[1] != g.size()[1]:
      jhamela += 1
      continue
    outputs = model(input_ids = inp, attention_mask=a, gazetteer = g)
    _, predicted_label = torch.max(outputs, dim=2)
    _, real = torch.max(label, dim=1)
    mac_f1.append(classification_report(predicted_label[0].to('cpu'),real.to('cpu'), output_dict = True, zero_division=0)['macro avg']['f1-score'])
print("MACRO F1 = ", sum(mac_f1) / (len(mac_f1)-jhamela))

  0%|          | 190/133114 [00:19<3:49:19,  9.66it/s]


KeyboardInterrupt: ignored

In [85]:
g.size()[1] != label.size()[0]

False

In [113]:
g.size()

torch.Size([1, 12, 13])

True

In [108]:
Gazetteer("১৯২৭ বেন্টলি ৪1⁄2 লিটার অনুরূপ ইঞ্জিন ডিজাইনের ছিল").size()

torch.Size([12, 13])

In [ ]:
from torch.nn import BCELoss

loss1 = BCELoss()
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for j in tqdm(range(df_train.shape[0])):
        optim.zero_grad()
        i,a,g, label = df_train['input_ids'][j].to(device) , df_train['attention_mask'][j].to(device) ,df_train['gazetteer'][j].to(device) ,df_train['Tag'][j].unsqueeze(0).to(device)
        outputs = model(input_ids = i, attention_mask=a, gazetteer = g)
        loss = loss1(outputs,label)
        if j % 100 == 0:
          print(loss)
        loss.backward()
        optim.step()

model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/15301 [00:00<34:25,  7.41it/s]

tensor(0.0388, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


  1%|          | 102/15301 [00:14<33:23,  7.59it/s]

tensor(0.0334, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


  1%|▏         | 202/15301 [00:31<37:25,  6.73it/s]

...

 56%|█████▌    | 8602/15301 [20:09<14:32,  7.68it/s]

tensor(0.0243, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


 57%|█████▋    | 8702/15301 [20:23<14:24,  7.63it/s]

tensor(0.0308, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


 57%|█████▋    | 8741/15301 [20:28<16:18,  6.70it/s]

In [ ]:
class NERDataset(Dataset):
  def __init__(self,df):
    self.input_ids = df['input_ids']
    self.attention_mask = df['attention_mask']
    self.gazetteer = df['gazetteer']
    self.label = df['Tag']
    self.n_samples = df.shape[0]

  def __getitem__(self, idx):
    return self.input_ids[idx].to(device),self.attention_mask[idx].to(device),self.gazetteer[idx].to(device),self.label[idx].to(device)
  
  def __len__(self):
    return self.n_samples
    

In [ ]:
train_dataset = NERDataset(df_train)

In [ ]:
train_dataset.n_samples

15301

In [97]:
import pickle

In [ ]:
train_loader = DataLoader(dataset = train_dataset, batch_size=1, shuffle=True)

In [ ]:
from torch.nn import BCELoss

loss = BCELoss()
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        i,a,g, label = batch
        outputs = model(input_ids = i, attention_mask=a, gazetteer = g)
        loss = loss(outputs,label)
        loss.backward()
        optim.step()

model.eval()

In [ ]:
from torch.nn import BCELoss

loss = BCELoss()
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        i,a,g, label = batch
        outputs = model(input_ids = i, attention_mask=a, gazetteer = g)
        loss = loss(outputs,label)
        loss.backward()
        optim.step()

model.eval()

ValueError: ignored

In [95]:
%cd /content/drive/MyDrive/Bracu/THESIS/Saved Models

/content/drive/MyDrive/Bracu/THESIS/Saved Models
